In [195]:
import os
import cv2
from SegTracker import SegTracker
from model_args import aot_args,sam_args,segtracker_args
from PIL import Image
from aot_tracker import _palette
import numpy as np
import torch
import imageio
import matplotlib.pyplot as plt
from scipy.ndimage import binary_dilation
import gc
def save_prediction(pred_mask,output_dir,file_name):
    save_mask = Image.fromarray(pred_mask.astype(np.uint8))
    save_mask = save_mask.convert(mode='P')
    save_mask.putpalette(_palette)
    save_mask.save(os.path.join(output_dir,file_name))
def colorize_mask(pred_mask):
    save_mask = Image.fromarray(pred_mask.astype(np.uint8))
    save_mask = save_mask.convert(mode='P')
    save_mask.putpalette(_palette)
    save_mask = save_mask.convert(mode='RGB')
    return np.array(save_mask)
def draw_mask(img, mask, alpha=0.7, id_countour=False):
    img_mask = np.zeros_like(img)
    img_mask = img
    if id_countour:
        # very slow ~ 1s per image
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[obj_ids!=0]

        for id in obj_ids:
            # Overlay color on  binary mask
            if id <= 255:
                color = _palette[id*3:id*3+3]
            else:
                color = [0,0,0]
            foreground = img * (1-alpha) + np.ones_like(img) * alpha * np.array(color)
            binary_mask = (mask == id)

            # Compose image
            img_mask[binary_mask] = foreground[binary_mask]

            countours = binary_dilation(binary_mask,iterations=1) ^ binary_mask
            img_mask[countours, :] = 0
    else:
        binary_mask = (mask!=0)
        countours = binary_dilation(binary_mask,iterations=1) ^ binary_mask
        foreground = img*(1-alpha)+colorize_mask(mask)*alpha
        img_mask[binary_mask] = foreground[binary_mask]
        img_mask[countours,:] = 0
        
    return img_mask.astype(img.dtype)

### Set parameters for input and output

In [196]:
video_name = 'all_activities_2'
input_dir = '/media/hannah.defazio/Padlock_DT/Data/notpublic/PTG/data/Coffee/coffee_recordings/extracted/'
output_dir = '/media/hannah.defazio/Padlock_DT/Data/notpublic/PTG/SAM_Track_Results'
exp = 'coffee-base'
io_args = {
    'input_video': f'{input_dir}/{video_name}/{video_name}.mp4',
    'output_mask_dir': f'{output_dir}/{exp}/{video_name}_masks', # save pred masks
    'output_video': f'{output_dir}/{exp}/{video_name}_{exp}_seg.mp4', # mask+frame vizualization, mp4 or avi, else the same as input video
    'output_gif': f'{output_dir}/{exp}/{video_name}_{exp}_seg.gif', # mask visualization
}

In [161]:
# The below code is useful for converting frames extracted from a ros_bag to a .mp4 file.
# It's not practical to use all of the extracted frames out of the box unless you know which frame
# shows the best example of objects to track!

# # set the directory containing the extracted images from a ROS bag
img_dir = f'{input_dir}/{video_name}/_extracted/images'

# # set the output video file name and codec
out_file = f'{input_dir}/{video_name}/{video_name}.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# # get the dimensions of the first image
img_path = os.path.join(img_dir, os.listdir(img_dir)[0])
img = cv2.imread(img_path)
height, width, channels = img.shape

# # create the VideoWriter object
out = cv2.VideoWriter(out_file, fourcc, 10, (width, height))

# # loop through the images and write them to the video
for img_name in sorted(os.listdir(img_dir)):
    img_path = os.path.join(img_dir, img_name)
    img = cv2.imread(img_path)
    out.write(img)

print(f'Saved {out_file}')
# # release the VideoWriter object and close the video file
out.release()

libpng error: Read Error


Saved /media/hannah.defazio/Padlock_DT/Data/notpublic/PTG/data/Coffee/coffee_recordings/extracted//all_activities_2/all_activities_2.mp4


### Tuning Grounding-DINO and SAM on the First Frame for Good Initialization

In [198]:
# choose good parameters in sam_args based on the first frame segmentation result
# other arguments can be modified in model_args.py
# note the object number limit is 255 by default, which requires < 10GB GPU memory with amp
sam_args['generator_args'] = {
        'points_per_side': 30,
        'pred_iou_thresh': 0.8,
        'stability_score_thresh': 0.9,
        'crop_n_layers': 1,
        'crop_n_points_downscale_factor': 2,
        'min_mask_region_area': 200,
    }

# Set Text args
'''
parameter:
    grounding_caption: Text prompt to detect objects in key-frames
    box_threshold: threshold for box 
    text_threshold: threshold for label(text)
    box_size_threshold: If the size ratio between the box and the frame is larger than the 
        box_size_threshold, the box will be ignored. This is used to filter out large boxes.
    reset_image: reset the image embeddings for SAM
'''
labels = ["hands", 'filter cone', 'liquid measuring cup', 'scale', 'glass container',
          'bag of coffee beans',
          'green timer', 'red thermometer', 'black coffee grinder',
          'coffee beans', 'paper filter', 'paper filter bag', 'black kettle'
         ]
grounding_caption = ". ".join(labels)
print(f'grounding caption: {grounding_caption}')
#grounding_caption = "paper piece. water jug with blue cap. black kettle. black kettle lid with cork handle. tea cup. red thermometer. green timer. metal spoon. glass container. jar of honey."
box_threshold, text_threshold, box_size_threshold, reset_image = 0.6, 0.2, 0.2, True

frame_idx = 30             # Modify this frame idx to a known exemplar frame, if necessary
cap = cv2.VideoCapture(io_args['input_video'])
cap.set(1, frame_idx)

segtracker = SegTracker(segtracker_args,sam_args,aot_args)
segtracker.restart_tracker()
with torch.cuda.amp.autocast():
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        pred_mask, annotated_frame = segtracker.detect_and_seg(frame, grounding_caption, box_threshold, text_threshold, box_size_threshold)
        torch.cuda.empty_cache()
        obj_ids = np.unique(pred_mask)
        obj_ids = obj_ids[obj_ids!=0]
        print("processed frame {}, obj_num {}".format(frame_idx,len(obj_ids)),end='\n')
        break
    cap.release()
    init_res = draw_mask(annotated_frame, pred_mask,id_countour=False)
    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(init_res)
    plt.show()
    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(colorize_mask(pred_mask))
    plt.show()

    del segtracker
    torch.cuda.empty_cache()
    gc.collect()

grounding caption: hands. filter cone. liquid measuring cup. scale. glass container. bag of coffee beans. green timer. red thermometer. black coffee grinder. coffee beans. paper filter. paper filter bag. black kettle
final text_encoder_type: bert-base-uncased
Model loaded from ./ckpt/groundingdino_swint_ogc.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])
SegTracker has been initialized


UnboundLocalError: local variable 'refined_merged_mask' referenced before assignment

### Generate Results for the Whole Video

In [165]:
# For every sam_gap frames, we use SAM to find new objects and add them for tracking
# larger sam_gap is faster but may not spot new objects in time
segtracker_args = {
    'sam_gap': 9999, # the interval to run sam to segment new objects
    'min_area': 200, # minimal mask area to add a new mask as a new object
    'max_obj_num': 255, # maximal object number to track in a video
    'min_new_obj_iou': 0.8, # the area of a new object in the background should > 80% 
}

# source video to segment
cap = cv2.VideoCapture(io_args['input_video'])
fps = cap.get(cv2.CAP_PROP_FPS)
# output masks
output_dir = io_args['output_mask_dir']
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
pred_list = []
masked_pred_list = []

torch.cuda.empty_cache()
gc.collect()
sam_gap = segtracker_args['sam_gap']
frame_idx = 0
segtracker = SegTracker(segtracker_args, sam_args, aot_args)
segtracker.restart_tracker()

with torch.cuda.amp.autocast():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        if frame_idx == 0:
            pred_mask, _ = segtracker.detect_and_seg(frame, grounding_caption, box_threshold, text_threshold, box_size_threshold, reset_image)
            # pred_mask = cv2.imread('./debug/first_frame_mask.png', 0)
            torch.cuda.empty_cache()
            gc.collect()
            segtracker.add_reference(frame, pred_mask)
        elif (frame_idx % sam_gap) == 0:
            seg_mask, _ = segtracker.detect_and_seg(frame, grounding_caption, box_threshold, text_threshold, box_size_threshold, reset_image)
            os.makedirs('./debug/seg_result', exist_ok=True)
            save_prediction(seg_mask, './debug/seg_result', str(frame_idx)+'.png')
            torch.cuda.empty_cache()
            gc.collect()
            track_mask = segtracker.track(frame)
            os.makedirs('./debug/aot_result', exist_ok=True)
            save_prediction(track_mask, './debug/aot_result', str(frame_idx)+'.png')
            # find new objects, and update tracker with new objects
            new_obj_mask = segtracker.find_new_objs(track_mask, seg_mask)
            if np.sum(new_obj_mask > 0) >  frame.shape[0] * frame.shape[1] * 0.4:
                new_obj_mask = np.zeros_like(new_obj_mask)
            save_prediction(new_obj_mask,output_dir,str(frame_idx)+'_new.png')
            pred_mask = track_mask + new_obj_mask
            # segtracker.restart_tracker()
            segtracker.add_reference(frame, pred_mask)
        else:
            pred_mask = segtracker.track(frame,update_memory=True)
        torch.cuda.empty_cache()
        gc.collect()
        
        save_prediction(pred_mask,output_dir,str(frame_idx)+'.png')
        # masked_frame = draw_mask(frame,pred_mask)
        # masked_pred_list.append(masked_frame)
        # plt.imshow(masked_frame)
        # plt.show() 
        
        pred_list.append(pred_mask)
        
        
        print("processed frame {}, obj_num {}".format(frame_idx,segtracker.get_obj_num()),end='\r')
        frame_idx += 1
    cap.release()
    print('\nfinished')

final text_encoder_type: bert-base-uncased
Model loaded from ./ckpt/groundingdino_swint_ogc.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])
SegTracker has been initialized


KeyboardInterrupt: 

### Save results for visualization

In [51]:
# draw pred mask on frame and save as a video
cap = cv2.VideoCapture(io_args['input_video'])
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

if io_args['input_video'][-3:]=='mp4':
    fourcc =  cv2.VideoWriter_fourcc(*"mp4v")
elif io_args['input_video'][-3:] == 'avi':
    fourcc =  cv2.VideoWriter_fourcc(*"MJPG")
    # fourcc = cv2.VideoWriter_fourcc(*"XVID")
else:
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
out = cv2.VideoWriter(io_args['output_video'], fourcc, fps, (width, height))

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    pred_mask = pred_list[frame_idx]
    masked_frame = draw_mask(frame,pred_mask)
    # masked_frame = masked_pred_list[frame_idx]
    masked_frame = cv2.cvtColor(masked_frame,cv2.COLOR_RGB2BGR)
    out.write(masked_frame)
    print('frame {} writed'.format(frame_idx),end='\r')
    frame_idx += 1
out.release()
cap.release()
print("\n{} saved".format(io_args['output_video']))
print('\nfinished')

frame 275 writed
/media/hannah.defazio/Padlock_DT/Data/notpublic/PTG/SAM_Track_Results/base-jar-honey-water/kitware_tea_video_12_trimmed_base-jar-honey-water_seg.mp4 saved

finished


In [52]:
# save colorized masks as a gif
imageio.mimsave(io_args['output_gif'],pred_list,duration=1000/fps)
print("{} saved".format(io_args['output_gif']))

/media/hannah.defazio/Padlock_DT/Data/notpublic/PTG/SAM_Track_Results/base-jar-honey-water/kitware_tea_video_12_trimmed_base-jar-honey-water_seg.gif saved


In [53]:
# manually release memory (after cuda out of memory)
del segtracker
torch.cuda.empty_cache()
gc.collect()

0